# Section 1: 環境セットアップ & データ探索

## 🎯 このセクションの目標
- TPCH_SF1データの構造を理解する
- 顧客の注文パターンを分析する
- チャーン（離反）の定義と分布を確認する

## 📊 シナリオ

> あなたは卸売業者のデータサイエンティストです。
> 
> 2024年末時点で、2年以上取引のある顧客の中から、
> 2025年に取引がなくなる（チャーンする）顧客を予測したいと考えています。


---
## 1.1 環境設定


In [ ]:
# 必要なライブラリのインポート
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *
import pandas as pd

# アクティブセッションの取得
session = get_active_session()

# コンテキストの確認
print(f"Database: {session.get_current_database()}")
print(f"Schema: {session.get_current_schema()}")
print(f"Warehouse: {session.get_current_warehouse()}")


In [ ]:
# 使用するデータベース・スキーマを設定
session.use_database("MLOPS_HOL_DB")
session.use_schema("PREP_DATA")
session.use_warehouse("MLOPS_HOL_PYTHON_WH")

print("✅ 環境設定完了！")


---
## 1.2 元データと前処理済みデータ

### 元データ: TPCH_SF1
- Snowflakeに標準搭載のベンチマークデータ
- 元データの日付範囲: 1992年〜1998年
- **このハンズオンでは +27年シフトして 2019年〜2025年 として扱います**

### 前処理済みデータ: PREP_DATA スキーマ
`01_prepare_training_data.sql` で以下のテーブルを作成済み:
- `CHURN_LABELS`: チャーンラベル（2024年アクティブ顧客 × チャーンフラグ）
- `CUSTOMER_FEATURES`: 顧客特徴量（17特徴量）
- `TRAINING_DATA`: 学習データセット


In [ ]:
# 前処理済みデータの規模を確認
tables = [
    ("CHURN_LABELS", "MLOPS_HOL_DB.PREP_DATA.CHURN_LABELS"),
    ("CUSTOMER_FEATURES", "MLOPS_HOL_DB.PREP_DATA.CUSTOMER_FEATURES"),
    ("TRAINING_DATA", "MLOPS_HOL_DB.PREP_DATA.TRAINING_DATA")
]

print("=== 前処理済みデータ規模 ===")
for name, path in tables:
    count = session.table(path).count()
    print(f"  {name}: {count:,} 件")


### 📋 学習データの確認


In [ ]:
# 学習データのスキーマ確認
training_df = session.table("MLOPS_HOL_DB.PREP_DATA.TRAINING_DATA")
print("=== TRAINING_DATAテーブルのカラム ===")
for field in training_df.schema.fields:
    print(f"  {field.name}: {field.datatype}")


In [ ]:
# サンプルデータの確認
training_df.show(5)


In [ ]:
# 市場セグメント別の顧客数
print("=== 市場セグメント別顧客数 ===")
training_df.group_by("MARKET_SEGMENT").count().order_by("COUNT", ascending=False).show()


### 📋 観測時点と特徴量日付の確認


In [ ]:
# 特徴量の基準日（観測時点）を確認
print("=== 観測時点 ===")
print("シナリオ: 2024年12月31日時点の特徴量で、2025年のチャーンを予測")
training_df.select("FEATURE_DATE").distinct().show()


In [ ]:
# 特徴量の統計サマリー
print("=== 主要特徴量の統計 ===")
training_df.select(
    "DAYS_SINCE_LAST_ORDER",
    "TOTAL_ORDER_COUNT", 
    "TOTAL_ORDER_AMOUNT",
    "AVG_ORDER_AMOUNT"
).describe().show()


In [ ]:
# 2024年の注文回数分布
print("=== 2024年の注文回数分布 ===")
training_df.with_column(
    "ORDER_BUCKET",
    F.when(F.col("ORDER_COUNT_2024") == 0, "0回")
     .when(F.col("ORDER_COUNT_2024") <= 2, "1-2回")
     .when(F.col("ORDER_COUNT_2024") <= 5, "3-5回")
     .otherwise("6回以上")
).group_by("ORDER_BUCKET") \
    .agg(F.count("*").alias("CUSTOMER_COUNT")) \
    .order_by("ORDER_BUCKET") \
    .show()


In [ ]:
# アカウント残高の分布
print("=== アカウント残高の分布 ===")
training_df.with_column(
    "BALANCE_BUCKET",
    F.when(F.col("ACCOUNT_BALANCE") < 0, "マイナス")
     .when(F.col("ACCOUNT_BALANCE") < 3000, "0-3000")
     .when(F.col("ACCOUNT_BALANCE") < 6000, "3000-6000")
     .otherwise("6000以上")
).group_by("BALANCE_BUCKET") \
    .agg(F.count("*").alias("CUSTOMER_COUNT")) \
    .order_by("BALANCE_BUCKET") \
    .show()


---
## 1.3 チャーンの定義と分析

**チャーンの定義**:
- 2024年に注文があった顧客のうち
- 2025年に注文がなかった顧客 = **チャーン（離反）**


In [ ]:
# 前処理済みのチャーンラベルを確認
churn_df = session.table("MLOPS_HOL_DB.PREP_DATA.CHURN_LABELS")
churn_df.show(10)


In [ ]:
# チャーン率の確認
churn_stats = churn_df.agg(
    F.count("*").alias("TOTAL_CUSTOMERS"),
    F.sum("IS_CHURNED").alias("CHURNED_CUSTOMERS")
).collect()[0]

total = churn_stats["TOTAL_CUSTOMERS"]
churned = churn_stats["CHURNED_CUSTOMERS"]
churn_rate = churned / total * 100

print("=" * 50)
print("📊 チャーン統計")
print("=" * 50)
print(f"  2024年のアクティブ顧客数: {total:,} 人")
print(f"  2025年にチャーンした顧客: {churned:,} 人")
print(f"  チャーン率: {churn_rate:.1f}%")
print("=" * 50)


---
## 1.4 顧客特徴量の確認

学習に使用する特徴量を確認します。


In [ ]:
# 特徴量データの確認
features_df = session.table("MLOPS_HOL_DB.PREP_DATA.CUSTOMER_FEATURES")
print(f"レコード数: {features_df.count():,}")
print(f"カラム数: {len(features_df.columns)}")


In [ ]:
# 特徴量一覧
print("=== 特徴量一覧 ===")
for col in features_df.columns:
    print(f"  - {col}")


In [ ]:
# サンプルデータ
features_df.show(5)


In [ ]:
# 特徴量の統計情報
print("=== 特徴量の統計 ===")
features_df.select(
    "DAYS_SINCE_LAST_ORDER",
    "TOTAL_ORDER_COUNT",
    "AVG_ORDER_AMOUNT",
    "ACCOUNT_BALANCE"
).describe().show()


---
## 1.5 チャーン顧客の特徴分析


In [ ]:
# 学習データの読み込み
training_df = session.table("MLOPS_HOL_DB.PREP_DATA.TRAINING_DATA")


In [ ]:
# チャーン vs アクティブの比較
print("=== チャーン状態別の特徴量平均 ===")
training_df.group_by("IS_CHURNED").agg(
    F.avg("DAYS_SINCE_LAST_ORDER").alias("AVG_RECENCY"),
    F.avg("TOTAL_ORDER_COUNT").alias("AVG_ORDER_COUNT"),
    F.avg("AVG_ORDER_AMOUNT").alias("AVG_ORDER_VALUE"),
    F.avg("ACCOUNT_BALANCE").alias("AVG_BALANCE")
).show()


In [ ]:
# 市場セグメント別チャーン率
print("=== 市場セグメント別チャーン率 ===")
training_df.group_by("MARKET_SEGMENT").agg(
    F.count("*").alias("CUSTOMER_COUNT"),
    F.sum("IS_CHURNED").alias("CHURNED_COUNT"),
    F.round(F.sum("IS_CHURNED") * 100 / F.count("*"), 2).alias("CHURN_RATE_PCT")
).order_by("CHURN_RATE_PCT", ascending=False).show()


In [ ]:
# 最終注文からの日数とチャーンの関係
print("=== 最終注文からの日数とチャーンの関係 ===")
training_df.with_column(
    "RECENCY_BUCKET",
    F.when(F.col("DAYS_SINCE_LAST_ORDER") <= 30, "0-30日")
     .when(F.col("DAYS_SINCE_LAST_ORDER") <= 90, "31-90日")
     .when(F.col("DAYS_SINCE_LAST_ORDER") <= 180, "91-180日")
     .otherwise("181日以上")
).group_by("RECENCY_BUCKET").agg(
    F.count("*").alias("COUNT"),
    F.round(F.sum("IS_CHURNED") * 100 / F.count("*"), 2).alias("CHURN_RATE_PCT")
).order_by("RECENCY_BUCKET").show()


---
## 1.6 発見した仮説

データ探索から、以下のような仮説が立てられます：

1. **最終注文からの日数（Recency）** が長いほどチャーンしやすい
2. **注文回数（Frequency）** が少ない顧客はチャーンリスクが高い
3. **市場セグメント** によってチャーン率に差がある可能性
4. **アカウント残高** とチャーンに関連がある可能性

次のセクションでは、これらの仮説を検証するための特徴量をFeature Storeに登録します。


---
## ✅ Section 1 完了！

### 学んだこと
- TPCH_SF1ベースの前処理済みデータの構造
- チャーンの定義（2024年アクティブ → 2025年に注文なし）
- チャーン率と顧客特徴量の関係

### 次のステップ
**Section 2: Feature Store** では、特徴量をFeature Storeに登録し、バージョン管理します。

➡️ `02_FEATURE_STORE` Notebookに進んでください
